In [ ]:
import imageio.v3 as iio
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
img_file_name = "GFP_06-DAPI.tif"
print("... reading image from file: " + img_file_name)

In [ ]:

def calculate_horizontal_gradient(image):
    # Calculate the horizontal gradient using finite differences
    gradient_x = np.zeros_like(image)
    gradient_x[:, :-1] = image[:, 1:] - image[:, :-1]

    return gradient_x

def calculate_vertical_gradient(image):
    # Calculate the vertical gradient using finite differences
    gradient_y = np.zeros_like(image)
    gradient_y[:-1, :] = image[1:, :] - image[:-1, :]

    return gradient_y

def display_image(image_array, title):
    plt.imshow(image_array, cmap='gray')
    plt.title(title)
    plt.axis('off')
    plt.show()

def normalize_to_uint8(image):
    imin = image.min()
    imax = image.max()
    image = ((image - imin)/(imax - imin) * 255).astype(np.uint8)
    
    return image

In [ ]:
def gaussian_1D(sigma, support_size):

    half_size = support_size // 2
    gaussian = np.zeros(support_size)

    for i in range(support_size):
        x = i - half_size
        #gf[i] = 1.0 / (np.sqrt(2 * np.pi) * sigma) * np.exp(-(x**2) / (2 * sigma**2))
        gaussian[i] = np.exp(-(x**2) / (2 * sigma**2))
        
    # Normalize the filter
    gaussian = gaussian / gaussian.sum()

    return gaussian

In [ ]:
original_image = iio.imread(img_file_name)

smoothing = False

if smoothing:
    gf = gaussian_1D(1.0, 3)
    print('Gaussian filter:', gf)
    height, width = original_image.shape
    smoothed_image = np.zeros(original_image.shape)
    
    for h in range(height):
        smoothed_image[h,:] = np.convolve(gf, original_image[h,:], mode='same')
    for w in range(width):
        smoothed_image[:,w] = np.convolve(gf, smoothed_image[:,w], mode='same')
    horizontal_gradient = calculate_horizontal_gradient(smoothed_image)
    vertical_gradient = calculate_vertical_gradient(smoothed_image)
else:
    horizontal_gradient = calculate_horizontal_gradient(original_image)
    vertical_gradient = calculate_vertical_gradient(original_image)

iio.imwrite('horizontal_gradient.png', horizontal_gradient)
iio.imwrite('vertical_gradient.png', vertical_gradient)


# Display the original image
display_image(original_image, "Original Image")

# Display the horizontal gradient
display_image(horizontal_gradient, "Horizontal Gradient")

display_image(vertical_gradient, "Vertical Gradient")

In [ ]:
gradient_magnitude = np.sqrt(horizontal_gradient**2 + vertical_gradient**2)
display_image(gradient_magnitude, "Gradient Magnitude")

gradient_magnitude_uint8 = normalize_to_uint8(gradient_magnitude)
iio.imwrite('gradient_magnitude.png', gradient_magnitude_uint8)

In [ ]:
print(gradient_magnitude.max())
idx = gradient_magnitude < 7.0
gradient_magnitude[idx] = 0
display_image(gradient_magnitude, "Thresholded Gradient Magnitude")